In [1]:

import pandas as pd
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


def prepare_sequences(notes, n_vocab, sequence_length = 100):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = sequence_length

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output, epochs=2, batch_size=128):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=epochs,
              batch_size=batch_size, callbacks=callbacks_list)




In [2]:
notes = []
log = []

annees = [2002]
notes_source = []
for x in annees:
    notesAnnees = pd.read_pickle(r'../data/results/notes_' + str(x))
    notes_source += notesAnnees
notes_source = notes_source[1000:2000]


# test avec entrée = toujours la même notes

In [16]:
notes = [notes_source[0]]*1000
n_vocab = len(set(notes))
print(notes[0:10])

['G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2']


In [17]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab)

print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/2
8/8 [==============================] - 16s 1s/step - loss: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 12s 1s/step - loss: 0.0000e+00
model trained


In [18]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 10 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]
    
print(prediction_output)

generating notes
['G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2']


conclusion: avec toujours la même note d'entrée, on renvoie la même en sortie: ouf!

# test avec entrée = alternance de 2 notes


In [3]:
notes = notes_source[0:2]*5000
n_vocab = len(set(notes))
print(notes[0:10])

['G#2', 'G2', 'G#2', 'G2', 'G#2', 'G2', 'G#2', 'G2', 'G#2', 'G2']


In [4]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab)

print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/2
78/78 [==============================] - 149s 2s/step - loss: 0.7048
Epoch 2/2
78/78 [==============================] - 137s 2s/step - loss: 0.6935
model trained


In [ ]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 10 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index/ float(n_vocab))
    pattern = pattern[1:len(pattern)]
    
print(prediction_output)

conclusion : il renvoie encore la même note: quel idiot!

tâchons de comprendre pourquoi...

Et si c'était les epochs qui sont pas assez nombreux? 

On va tester pour 40 au lieu de 2, toutes choses étant égales par ailleurs

In [96]:
print('training model')
train(model, network_input, network_output, epochs=10, batch_size=128)

print('model trained')

training model
Epoch 1/10
8/8 [==============================] - 12s 2s/step - loss: 0.6936
Epoch 2/10
8/8 [==============================] - 13s 2s/step - loss: 0.6951
Epoch 3/10
8/8 [==============================] - 12s 1s/step - loss: 0.6947
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.6952
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.6974
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 1.0045
Epoch 7/10
8/8 [==============================] - 12s 1s/step - loss: 0.6947
Epoch 8/10
8/8 [==============================] - 12s 1s/step - loss: 0.6947
Epoch 9/10
8/8 [==============================] - 12s 1s/step - loss: 0.6963
Epoch 10/10
8/8 [==============================] - 12s 1s/step - loss: 0.7354
model trained


In [100]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 10 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index/ float(n_vocab))
    pattern = pattern[1:len(pattern)]
    
print(prediction_output)

generating notes
['G2', 'G#2', 'G2', 'G#2', 'G2', 'G#2', 'G2', 'G#2', 'G2', 'G#2']


conclusion: c'est pas un pb d'epochs

Peut être la sequence length?

On va passer à 10...

In [110]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab, sequence_length=100)

print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output, epochs=10, batch_size=128)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/10
57/57 [==============================] - 32s 484ms/step - loss: 0.7071
Epoch 2/10
57/57 [==============================] - 27s 479ms/step - loss: 0.7737
Epoch 3/10
57/57 [==============================] - 28s 493ms/step - loss: 0.7089
Epoch 4/10


KeyboardInterrupt: 

In [108]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []

# generate 10 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    print(prediction_input)
    prediction = model.predict(prediction_input, verbose=0)
    print(prediction)
    index = numpy.argmax(prediction)
    print(index)
    result = int_to_note[index]
    print(result)
    prediction_output.append(result)
    print(prediction_output)
    pattern = numpy.append(pattern,index/ float(n_vocab))
    print(pattern)
    pattern = pattern[1:len(pattern)]
    print(pattern)
    print("----")

print(prediction_output)



generating notes
[[[0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]]]
[[9.985313e-06 9.999900e-01]]
1
G2
['G2']
[0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.
 0.5 0.  0.5]
[0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5
 0.  0.5]
----
[[[0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]]]
[[9.9997783e-01 2.2214997e-05]]
0
G#2
['G2', 'G#2']
[0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5
 0.  0.5 0. ]
[0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.  0.5 0.
 0.5 0. ]
----
[[[0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]
  [0.5]
  [0. ]]]
[[9.985313e-06 9.999900e-01]]
1
G2
['G2', 'G#2', 'G2']


Conclusion: victoire pour le peuple